<a href="https://colab.research.google.com/github/hemu2014/python-ML/blob/main/%E6%9C%BA%E5%99%A8%E5%AD%A6%E4%B9%A0-%E7%AE%97%E6%B3%95%E5%88%86%E7%B1%BB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import tensorflow as tf
import pandas as pd

# 数据加载与预处理
csv_columns = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
train_path = tf.keras.utils.get_file('iris_training.csv', "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file('iris_test.csv', "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=csv_columns, header=0)
test = pd.read_csv(test_path, names=csv_columns, header=0)
train_y = train.pop("Species")
test_y = test.pop("Species")

# 定义输入函数（显式指定数据类型）
def input_fn(features, labels, training=True, batch_size=32):
    #features_tensor = tf.convert_to_tensor(features.values, dtype=tf.float32)
    #labels_tensor = tf.convert_to_tensor(labels.values, dtype=tf.int32)
    #dataset = tf.data.Dataset.from_tensor_slices((features_tensor, labels_tensor))
    dataset = tf.data.Dataset.from_tensor_slices((features.values, labels.values))
    if training:
        dataset = dataset.shuffle(1000).repeat()
    return dataset.batch(batch_size)

# 创建数据集
train_set = input_fn(train, train_y, training=True)
test_set = input_fn(test, test_y, training=False)

# 检查数据集格式
print("训练集元素格式:", train_set.element_spec)

训练集元素格式: (TensorSpec(shape=(None, 4), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))


In [14]:
# 构建归一化层
normalizer = tf.keras.layers.Normalization(axis=-1, input_shape=[4])
normalizer.adapt(train.values)  # 确保适配正确的数据

# 构建模型
model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(30, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

# 编译模型
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 计算合理的 steps_per_epoch
batch_size = 32
num_train_samples = len(train)
steps_per_epoch = (num_train_samples + batch_size - 1) // batch_size

# 训练模型
model.fit(
    train_set,
    epochs=50,
    steps_per_epoch=steps_per_epoch,
    #validation_data=test_set,
    verbose=2
)

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 - 1s - 297ms/step - accuracy: 0.3438 - loss: 1.0582
Epoch 2/50
4/4 - 0s - 3ms/step - accuracy: 0.3438 - loss: 1.0010
Epoch 3/50
4/4 - 0s - 3ms/step - accuracy: 0.5000 - loss: 0.9730
Epoch 4/50
4/4 - 0s - 3ms/step - accuracy: 0.6172 - loss: 0.9214
Epoch 5/50
4/4 - 0s - 3ms/step - accuracy: 0.7188 - loss: 0.8788
Epoch 6/50
4/4 - 0s - 3ms/step - accuracy: 0.8047 - loss: 0.8221
Epoch 7/50
4/4 - 0s - 3ms/step - accuracy: 0.7031 - loss: 0.8197
Epoch 8/50
4/4 - 0s - 3ms/step - accuracy: 0.8047 - loss: 0.7752
Epoch 9/50
4/4 - 0s - 3ms/step - accuracy: 0.7734 - loss: 0.7543
Epoch 10/50
4/4 - 0s - 3ms/step - accuracy: 0.7578 - loss: 0.7584
Epoch 11/50
4/4 - 0s - 3ms/step - accuracy: 0.8203 - loss: 0.6833
Epoch 12/50
4/4 - 0s - 3ms/step - accuracy: 0.7891 - loss: 0.6911
Epoch 13/50
4/4 - 0s - 3ms/step - accuracy: 0.7969 - loss: 0.6897
Epoch 14/50
4/4 - 0s - 3ms/step - accuracy: 0.8281 - loss: 0.6126
Epoch 15/50
4/4 - 0s - 3ms/step - accuracy: 0.7969 - loss: 0.6480
Epoch 16/50
4/4 - 0s - 3ms/s

In [15]:
loss, accuracy = model.evaluate(test_set, verbose=2)                # ②
print(f'\nTest set accuracy: {accuracy:.3f}')

1/1 - 0s - 164ms/step - accuracy: 0.9333 - loss: 0.2800

Test set accuracy: 0.933


In [16]:
import numpy as np
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

# 6. 预测逻辑示例
# 6.1 单个样本预测
sample = np.array([[5.1, 3.3, 1.7, 0.5]], dtype=np.float32)
probs  = model.predict(sample)              # 形状 (1,3)
pred   = np.argmax(probs, axis=1)[0]       # 得到类别索引 0/1/2
print(f"Sample features: {sample.flatten().tolist()}")
print(f"Predicted probabilities: {probs.flatten().tolist()}")
print(f"Predicted class index: {pred} -> {SPECIES[pred]}")

# 6.2 批量样本预测（以测试集前5条为例）
batch_samples = test.values[:5, :4].astype(np.float32)
batch_probs   = model.predict(batch_samples)
batch_pred    = np.argmax(batch_probs, axis=1)
for i, feat in enumerate(batch_samples):
    print(f"\nSample {i+1}: {feat.tolist()}")
    print(f"  True label:    {SPECIES[test_y.iloc[i]]}")
    print(f"  Pred probabilities: {batch_probs[i].tolist()}")
    print(f"  Predicted class:    {SPECIES[batch_pred[i]]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
Sample features: [5.099999904632568, 3.299999952316284, 1.7000000476837158, 0.5]
Predicted probabilities: [0.9073820114135742, 0.034905724227428436, 0.05771222338080406]
Predicted class index: 0 -> Setosa
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step

Sample 1: [5.900000095367432, 3.0, 4.199999809265137, 1.5]
  True label:    Versicolor
  Pred probabilities: [0.05209425091743469, 0.5164095759391785, 0.43149617314338684]
  Predicted class:    Versicolor

Sample 2: [6.900000095367432, 3.0999999046325684, 5.400000095367432, 2.0999999046325684]
  True label:    Virginica
  Pred probabilities: [0.0018518163124099374, 0.08850055187940598, 0.9096476435661316]
  Predicted class:    Virginica

Sample 3: [5.099999904632568, 3.299999952316284, 1.7000000476837158, 0.5]
  True label:    Setosa
  Pred probabilities: [0.9073820114135742, 0.03490573540329933, 0.05771224945783615]
  Predicted class:    Setosa

Sample 4: [6.0, 3.4000000953674316, 4.5, 1.600000023841858]
  T